# 04 - Train DistilBERT Multi-label Classifier

## Goal

Train DistilBERT for multi-label classification on title+abstract. We'll use BCEWithLogits, monitor micro/macro F1, and tune threshold on the validation set.


## Why Multi-label Classification?

Each paper can have multiple study characteristics:
- An **RCT** is also **Human**
- A **Systematic Review** might be a **MetaAnalysis**
- Some studies combine **Animal** + **InVitro**

We use **binary cross-entropy** (BCE) for each label independently.


In [ ]:
# TODO: Import libraries
# Hint: import pandas as pd, numpy as np
# from pathlib import Path
# from datasets import Dataset
# from transformers import (
#     AutoTokenizer, AutoModelForSequenceClassification,
#     TrainingArguments, Trainer
# )
# import evaluate
# import torch


In [ ]:
# TODO: Define canonical labels (fixed order!)
# Hint: LABELS = [
#     'SystematicReview', 'MetaAnalysis', 'RCT', 'ClinicalTrial',
#     'Cohort', 'CaseControl', 'CaseReport', 'InVitro', 'Animal', 'Human'
# ]
# NUM_LABELS = len(LABELS)
# print(f"Training for {NUM_LABELS} labels: {LABELS}")


## Load Splits & Build HF Datasets


In [ ]:
# TODO: Load splits & build HF Datasets
# Hint: train_df = pd.read_parquet('../data/processed/train.parquet')
#       val_df = pd.read_parquet('../data/processed/val.parquet')
#       test_df = pd.read_parquet('../data/processed/test.parquet')
# 
# # Concatenate title + abstract; truncate to 2k chars
# train_df['text'] = (train_df['title'] + ' ' + train_df['abstract']).str[:2000]
# val_df['text'] = (val_df['title'] + ' ' + val_df['abstract']).str[:2000]
# test_df['text'] = (test_df['title'] + ' ' + test_df['abstract']).str[:2000]
# 
# print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")


## Binarize Labels

Convert list of labels → multi-hot binary vector.


In [ ]:
# TODO: Binarize labels
# Hint: def binarize_labels(labels_list):
#     vec = np.zeros(NUM_LABELS, dtype=float)
#     for label in labels_list:
#         if label in LABELS:
#             vec[LABELS.index(label)] = 1.0
#     return vec.tolist()
# 
# train_df['label_vec'] = train_df['labels'].apply(binarize_labels)
# val_df['label_vec'] = val_df['labels'].apply(binarize_labels)
# test_df['label_vec'] = test_df['labels'].apply(binarize_labels)


In [ ]:
# TODO: Convert to HF Dataset
# Hint: train_dataset = Dataset.from_pandas(train_df[['text', 'label_vec']])
#       val_dataset = Dataset.from_pandas(val_df[['text', 'label_vec']])
#       test_dataset = Dataset.from_pandas(test_df[['text', 'label_vec']])


## Tokenizer & Encoding


In [ ]:
# TODO: Tokenizer & encoding
# Hint: tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
# 
# def tokenize_function(examples):
#     return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
# 
# train_dataset = train_dataset.map(tokenize_function, batched=True)
# val_dataset = val_dataset.map(tokenize_function, batched=True)
# test_dataset = test_dataset.map(tokenize_function, batched=True)
# 
# # Rename label_vec → labels for Trainer
# train_dataset = train_dataset.rename_column('label_vec', 'labels')
# val_dataset = val_dataset.rename_column('label_vec', 'labels')
# test_dataset = test_dataset.rename_column('label_vec', 'labels')
# 
# train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
# val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
# test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])


## Model Initialization


In [ ]:
# TODO: Model init
# Hint: model = AutoModelForSequenceClassification.from_pretrained(
#     'distilbert-base-uncased',
#     num_labels=NUM_LABELS,
#     problem_type='multi_label_classification'
# )


## Metrics Function


In [ ]:
# TODO: Metrics function
# Hint: from sklearn.metrics import precision_recall_fscore_support
# 
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     probs = torch.sigmoid(torch.tensor(logits)).numpy()
#     preds = (probs > 0.5).astype(int)
#     
#     precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
#         labels, preds, average='micro', zero_division=0
#     )
#     precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
#         labels, preds, average='macro', zero_division=0
#     )
#     
#     return {
#         'precision_micro': precision_micro,
#         'recall_micro': recall_micro,
#         'f1_micro': f1_micro,
#         'precision_macro': precision_macro,
#         'recall_macro': recall_macro,
#         'f1_macro': f1_macro
#     }


## Training Arguments & Trainer


In [ ]:
# TODO: TrainingArguments
# Hint: training_args = TrainingArguments(
#     output_dir='../artifacts/model/checkpoints',
#     eval_strategy='epoch',
#     save_strategy='epoch',
#     learning_rate=5e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     warmup_ratio=0.1,
#     logging_dir='../artifacts/logs',
#     logging_steps=100,
#     load_best_model_at_end=True,
#     metric_for_best_model='f1_micro',
#     greater_is_better=True,
#     save_total_limit=2
# )


In [ ]:
# TODO: Train & save
# Hint: trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics
# )
# 
# trainer.train()
# 
# # Save best model
# Path('../artifacts/model/best').mkdir(parents=True, exist_ok=True)
# trainer.save_model('../artifacts/model/best')
# tokenizer.save_pretrained('../artifacts/model/best')
# print("✅ Model saved to ../artifacts/model/best")


## Recommendations

- If rare labels underperform, try **class weights** or **focal loss** (stretch goal)
- Track **ROC/PR curves** on val to tune per-label thresholds later
- Monitor for **overfitting:** val metrics should not degrade while train improves

## 🧘 Reflection Log

**What did you learn in this session?**
- 

**What challenges did you encounter?**
- 

**How will this improve Periospot AI?**
- 
